# Natural Language Processing

In [2]:
# Importing TF and checking the version
import tensorflow as tf

print(tf.__version__)

2.10.1


In [3]:
# Importing helper functions
from DanielBourke_HelperFunctions import create_tensorboard_callback, plot_loss_curves, compare_historys

### Analysing text dataset

In [4]:
# Loading data
import pandas as pd

train_df = pd.read_csv("NLP_text/train.csv")
test_df = pd.read_csv("NLP_text/test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffling training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# Checking test dataframe
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# Checking number of training records
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# Checking total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Visualising random samples
import random

random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[["text", "target"]][random_index : random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}")
    print("---")

Target: 1 (real disaster)
Text:
Myanmar Flooding: Heavy monsoon rains during the month of July have caused flooding flash floods and landsli... http://t.co/9TG7A5OqFP
---
Target: 0 (not real disaster)
Text:
I've just watched episode S01E09 of Bloody Monday! http://t.co/vRptHvPymt #bloodymonday #tvshowtime http://t.co/NkKvknBvOz
---
Target: 0 (not real disaster)
Text:
@whvholst @leashless And this is a structural problem rather than just a failure of competence by traditional soc democratic parties.
---
Target: 0 (not real disaster)
Text:
I SCREAMED 'WHATS A CHONCe' http://t.co/GXYivsWki7
---
Target: 0 (not real disaster)
Text:
@FaZe_Rain all hail the Cloud
---


### Creating validation data

In [10]:
# Splitting dataset
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1, # Allocating 10% to validation data
    random_state=42
)

In [11]:
# Checking dataset length
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [12]:
# Checking the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### Converting text to numbers

In [13]:
# Using text vectorisation
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False
)

In [14]:
# Finding the average number of tokens
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [15]:
# Setting up text vectorisation variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [16]:
# Fitting text vectorisation to the training dataset
text_vectorizer.adapt(train_sentences)

In [17]:
# Create a sample sentnence and tokenise it
sample_sentence = "There is a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [18]:
# Tokenising random sentence from the training set
random_sentence = random.choice(train_sentences)
print(
    f"Original text:\n{random_sentence},\ntokenised version:\n{text_vectorizer([random_sentence])}"
)

Original text:
Las Vegas in top 5 cities for red-light running fatalities - News3LV http://t.co/eXdbcx4gCR,
tokenised version:
[[3691 3284    4  212  180 2523   10 2752  876  301    1    1    0    0
     0]]


In [19]:
# Getting unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

### Creating Embedding layer

In [20]:
# Defining the layer
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    input_length=max_length
)
embedding

In [21]:
# Get a random sentence
random_sentence = random.choice(train_sentences)
print(
    f"Original text:\n{random_sentence},\nembedded version:"
)

# Embed random sentence
embed_sentence = embedding(text_vectorizer([random_sentence]))
embed_sentence

Original text:
One man fatally shot another wounded on Vermont Street #Buffalo - http://t.co/KakY4mpCO4,
embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00968846,  0.01396506, -0.0358798 , ...,  0.00607602,
         -0.03935768, -0.03889903],
        [-0.04239941, -0.03526977,  0.02661167, ..., -0.02152009,
          0.00812198,  0.04224873],
        [-0.01915747, -0.02273409, -0.03975364, ..., -0.00230503,
         -0.03034551, -0.02134173],
        ...,
        [-0.02363898, -0.03911495,  0.00859547, ..., -0.01244408,
         -0.04660244,  0.04752853],
        [-0.02363898, -0.03911495,  0.00859547, ..., -0.01244408,
         -0.04660244,  0.04752853],
        [-0.02363898, -0.03911495,  0.00859547, ..., -0.01244408,
         -0.04660244,  0.04752853]]], dtype=float32)>

In [22]:
# Checking single token's embedding
embed_sentence[0][0], embed_sentence[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00968846,  0.01396506, -0.0358798 , -0.00579258,  0.02711185,
         0.03818165,  0.02656038, -0.0049054 , -0.03423984, -0.03988526,
         0.0453656 ,  0.03061196,  0.03256241,  0.03272395, -0.01565869,
         0.01551572,  0.03512523,  0.00894897, -0.03341515,  0.04608497,
        -0.03900247, -0.00272442, -0.03066324, -0.00293616, -0.00038679,
        -0.01755118,  0.00121274, -0.03629506,  0.0486945 , -0.02841423,
        -0.01149614,  0.00770857, -0.01488087,  0.00182436, -0.02239796,
        -0.02087336, -0.04782781,  0.03007961,  0.02408793,  0.01827877,
         0.03313423, -0.02629081,  0.02351326, -0.01033636,  0.04921806,
        -0.02374013, -0.04042191,  0.01003786,  0.03435279, -0.03145609,
         0.02293422, -0.00025246, -0.04681561,  0.03124117,  0.02302232,
         0.04082045,  0.00339414, -0.0206479 , -0.04246751,  0.02989384,
        -0.02727183, -0.04942685,  0.00523769, -0.0419889 ,  0.02113746,
  

### Building base model

In [23]:
# Using SKLearn to build base model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenisation and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers
    ("clf", MultinomialNB()) # Model the text
])

# Fit the pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
# Evaluating baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Achieved accuracy: {baseline_score * 100:.2f}%")

Achieved accuracy: 79.27%


In [25]:
# Making predictions
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

### Function to evaluate model performance

In [26]:
# Importing SKLearn functions
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Developing function to evaluate accuracy, precision, recall and F1 scor
def calculate_results(y_true, y_pred):
    """
    Evaluate binary classification model
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and F1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1
    }
    return model_results

In [27]:
# Getting baseline results
baseline_results = calculate_results(
    y_true=val_labels,
    y_pred=baseline_predictions
)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: a simple dense model

In [28]:
# Creating tensorboard callback
from DanielBourke_HelperFunctions import create_tensorboard_callback

# Creating log directory
SAVE_DIR = "model_logs"

In [29]:
# Building model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype=tf.string)
x = text_vectorizer(inputs) # Turn the input text into numbers
x = embedding(x) # Create embedding of numberised inputs
x = layers.GlobalAveragePooling1D()(x) # Condence the feature vector
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [30]:
# Getting model summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [31]:
# Compiling model
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [32]:
# Fitting the model
history_1 = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_1_dense"
    )]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20230113-184557
Epoch 1/5
215/215 [==============================] - 4s 17ms/step - loss: 0.6140 - accuracy: 0.6967 - val_loss: 0.5370 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4437 - accuracy: 0.8187 - val_loss: 0.4720 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3488 - accuracy: 0.8597 - val_loss: 0.4568 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2870 - accuracy: 0.8863 - val_loss: 0.4637 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2395 - accuracy: 0.9083 - val_loss: 0.4765 - val_accuracy: 0.7808


In [33]:
# Checking the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4765 - accuracy: 0.7808


[0.4765395522117615, 0.7808399200439453]

In [34]:
# Making predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [35]:
# First 10 predictions
model_1_pred_probs[:10]

array([[0.43406558],
       [0.795222  ],
       [0.9969521 ],
       [0.13736199],
       [0.13006048],
       [0.9461528 ],
       [0.9262485 ],
       [0.99350214],
       [0.9615118 ],
       [0.31837198]], dtype=float32)

In [36]:
# Converting model predictions to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [37]:
# Calculating model 1 results
model_1_results = calculate_results(
    y_true=val_labels,
    y_pred=model_1_preds
)
model_1_results

{'accuracy': 78.08398950131233,
 'precision': 0.783783808499639,
 'recall': 0.7808398950131233,
 'f1': 0.7783998521836788}

In [38]:
# Retrieving baseline results
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [39]:
# Comparing model results
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualising learned embeddings

In [40]:
# Getting vocabulary from the text vectorisation layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [41]:
# Checking model 1 details
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [42]:
# Getting the weights
embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights, embed_weights[0].shape

([array([[-0.00769289, -0.05575653, -0.00751088, ..., -0.02065393,
          -0.06312703,  0.06436224],
         [ 0.0389185 ,  0.00975791,  0.02301645, ...,  0.01619697,
          -0.02994848, -0.03841233],
         [-0.03964905, -0.02566376, -0.00914194, ..., -0.02870046,
           0.02779888,  0.00269444],
         ...,
         [-0.01721511,  0.02341035, -0.02402272, ..., -0.01825237,
           0.0126978 , -0.04923532],
         [ 0.05241601, -0.06995192, -0.03782928, ..., -0.04998912,
          -0.02318824,  0.02081083],
         [ 0.03481236, -0.09860085, -0.09675076, ..., -0.01369869,
          -0.03498197,  0.04565519]], dtype=float32)],
 (10000, 128))

In [43]:
# Creating embedding files (sample from TF Word Embedding documentation)
import io

out_v = io.open("vectors.tsv", "w", encoding="utf-8")
out_m = io.open("metadata.tsv", "w", encoding="utf-8")

for index, word in enumerate(words_in_vocab):
    if index == 0:
        continue # Skip 0, as it's a padding
    vec = embed_weights[0][index]
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

Generated vectors.tsv and metadata.tsv can be uploaded into Tensorflow Embedding Projector tool at https://projector.tensorflow.org/ to visualise weights in 3D space.

### Model 2 - RNN (Recurrent Neural Network) with LSTM (Long Short-Term Memory)

In [44]:
# CReating an RNN model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x) # Sequences required when you stack LSTM layers
# print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [45]:
# Checking model's summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [46]:
# Compiling the model
model_2.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [47]:
# Fittting the model
history_2 = model_2.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_2_LSTM"
    )]
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230113-184617
Epoch 1/5
215/215 [==============================] - 7s 19ms/step - loss: 0.2174 - accuracy: 0.9212 - val_loss: 0.6148 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1570 - accuracy: 0.9400 - val_loss: 0.6301 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1285 - accuracy: 0.9505 - val_loss: 0.6412 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1042 - accuracy: 0.9585 - val_loss: 0.7806 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0821 - accuracy: 0.9677 - val_loss: 1.0548 - val_accuracy: 0.7795


In [48]:
# Meking predictions
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.5964566e-02],
       [9.4831741e-01],
       [9.9968791e-01],
       [1.8260200e-01],
       [1.4974400e-04],
       [9.9891931e-01],
       [9.2977363e-01],
       [9.9983412e-01],
       [9.9968874e-01],
       [6.6843361e-01]], dtype=float32)

In [49]:
# Converting pred probs to label format
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [50]:
#Calculate model 2 results
model_2_results = calculate_results(
    y_true=val_labels,
    y_pred=model_2_preds
)
model_2_results

{'accuracy': 77.95275590551181,
 'precision': 0.7797634583030512,
 'recall': 0.7795275590551181,
 'f1': 0.7784163768415738}

### Model 3 - RNN with GRU (gated recurrent unit)

In [51]:
# Building the model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, return_sequences=True)(x) # Sequences, if GRU are stuck on each other
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x) # If we want to consolidate GRU layer with sequences
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [52]:
# Getting model summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [53]:
# Compiling the model
model_3.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [54]:
# Fitting the model
history_3 = model_3.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_3_GRU"
    )]
)

Saving TensorBoard log files to: model_logs/model_3_GRU/20230113-184639
Epoch 1/5
215/215 [==============================] - 7s 18ms/step - loss: 0.1575 - accuracy: 0.9393 - val_loss: 0.7278 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0828 - accuracy: 0.9695 - val_loss: 0.7587 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0673 - accuracy: 0.9739 - val_loss: 0.8129 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0616 - accuracy: 0.9747 - val_loss: 1.3231 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0515 - accuracy: 0.9764 - val_loss: 1.1933 - val_accuracy: 0.7717


In [55]:
# Meking predictions
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs

24/24 [==============================] - 1s 4ms/step


array([[4.95823042e-04],
       [6.76481485e-01],
       [9.99863863e-01],
       [1.01334974e-01],
       [7.28685191e-05],
       [9.99664783e-01],
       [8.81450176e-01],
       [9.99928951e-01],
       [9.99872983e-01],
       [7.25330830e-01],
       [2.27102675e-04],
       [7.77818561e-01],
       [1.62134995e-04],
       [1.95992500e-01],
       [6.43431049e-05],
       [1.33154762e-03],
       [5.01151837e-04],
       [1.76692920e-04],
       [8.54680687e-03],
       [9.99755621e-01],
       [9.99021292e-01],
       [3.96493997e-05],
       [9.99305308e-01],
       [1.75265118e-03],
       [9.99880791e-01],
       [9.99903202e-01],
       [9.19120561e-04],
       [5.27382188e-04],
       [1.56360591e-04],
       [2.04580560e-01],
       [9.98946130e-01],
       [6.88185915e-04],
       [3.36996443e-03],
       [1.66688778e-03],
       [2.31705040e-01],
       [2.85641432e-01],
       [9.99790013e-01],
       [2.04382196e-01],
       [2.57885996e-02],
       [9.99939203e-01],


In [56]:
# Converting model 3 pred probs to label format
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [57]:
# Calculate model 3 results
model_3_results = calculate_results(
    y_true=val_labels,
    y_pred=model_3_preds
)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.7751367943413853,
 'recall': 0.7716535433070866,
 'f1': 0.7687482528669142}

### Model 4 - Bidirectional RNN

In [60]:
# Create a model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidir")

In [61]:
# Checking model details
model_4.summary()

Model: "model_4_bidir"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
N

In [62]:
# Compiling the model
model_4.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [63]:
# Fitting the model
history_5 = model_4.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_4_bidir"
    )]
)

Saving TensorBoard log files to: model_logs/model_4_bidir/20230113-221315
Epoch 1/5
215/215 [==============================] - 14s 38ms/step - loss: 0.1065 - accuracy: 0.9692 - val_loss: 0.9630 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 11s 49ms/step - loss: 0.0535 - accuracy: 0.9768 - val_loss: 1.1336 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 12s 55ms/step - loss: 0.0430 - accuracy: 0.9803 - val_loss: 1.2939 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 11s 51ms/step - loss: 0.0425 - accuracy: 0.9803 - val_loss: 1.4314 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 12s 56ms/step - loss: 0.0409 - accuracy: 0.9794 - val_loss: 1.3663 - val_accuracy: 0.7625


In [64]:
# Making predictions
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 2s 8ms/step


array([[1.3874387e-03],
       [8.2908392e-01],
       [9.9998486e-01],
       [3.0986142e-01],
       [4.9365179e-05],
       [9.9988669e-01],
       [9.8795718e-01],
       [9.9998981e-01],
       [9.9997860e-01],
       [9.9698669e-01]], dtype=float32)

In [65]:
# Adjusting model's probs to the label format
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [67]:
# Checking results of predictions
model_4_results = calculate_results(
    y_true=val_labels,
    y_pred=model_4_preds
)
model_4_results

{'accuracy': 76.24671916010499,
 'precision': 0.7626600366229389,
 'recall': 0.7624671916010499,
 'f1': 0.7611077640788777}